### Importing Modules

In [10]:
import pandas as pd
import numpy as np
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
en_stops = set(stopwords.words('english'))

wnl = WordNetLemmatizer()

### Defining Functions to manipulate CSV file into DataFrame

In [9]:
# Get a pandas DataFrame object of all the data in the csv file:
df = pd.read_csv('Challenge1_Training_Scenarios.csv')
df = df[['scenario','danger_level']]

# splits the string and returns the age and remaining text
def str_split(var):
    df1 = var.replace('-',' ')
    df1 = df1.replace(',',' ')
    df1 = df1.split(' ')
    df2 = df1[1]
    df1 = ' '.join(df1[4:])
    return(df2,df1)

# removes stop words from the text. May be useful. Received from Jaimie
def removeStopWords(text):
    returnText = ''
    for w in text.split(' '):
        if w not in en_stops and len(w)>0:
            returnText += ' ' + w
    returnText = returnText.strip()
    return returnText

# removes punctuation/etc from the DF
def nlpCleanup(df, columnName):
    df[columnName] = df[columnName].str.replace('\d+', '',regex=True) # for digits
#     df[columnName] = df[columnName].str.replace(r'(\b\w{1,2}\b)', '',regex=True) # for word length lt 2
    df[columnName] = df[columnName].str.replace('[^\w\s]', '',regex=True) # for punctuation 
#     df[columnName] = df[columnName].apply(removeStopWords)
    # for i, row in enumerate(df[columnName]):
    #     text = removeStopWords(row)
    #     df[columnName].loc[i] = text
    #     print(text)
    return df

# find position of the verb, splits string at the verb to create two string cells
def verb_pos_grab(df_string):
    tag_pos_string = nltk.pos_tag(word_tokenize(df_string))
    
    new_list = []
    word_list = []
    v1, v2, before = 0, 0, ''
    
    for i,j in tag_pos_string:
        new_list.append(j)
        word_list.append(i)
  
    if 'VB' in new_list:
        v1 = new_list.index('VB')
    if 'VBP' in new_list:
        v2 = new_list.index('VBP')
        
        
    if v1 == 0:
        verb_pos = v2
    elif v2 == 0:
        verb_pos = v1
    elif v1 <= v2:
        verb_pos  = v1
    elif v2 < v1:
        verb_pos = v2
    else:
        verb_pos = 0

#     print(new_list)
#     print(word_list)
#     print(v1)
#     print(v2)
#     print(verb_pos)

    before  = ' '.join(word_list[:verb_pos])
    new_string = ' '.join(word_list[verb_pos:])
    return(before, new_string)
    
    
'''
NEED TO FIGURE OUT HOW TO INDEX MULTIPLE VERB TYPE POSITIONS
'''

# lemmatizes the full string
def lemmatize(df_var):
    new_token = nltk.word_tokenize(df_var)
    token_1 = []
    for i in new_token:
        token_1.append(wnl.lemmatize(i, pos = 'v'))
    final = ' '.join(token_1)
    return(final)
 




### DataFrame Manipulation (Incomplete)

I have started to split each text by age, "description" of the person, and the action. 

1. I still need to figure out how to split the sentence multiple times so that multiple actions can be documented
2. I also need to work on verb split since I have not yet included adverbs
3. I need to make a scale factor for the age that can be implemented to the final risk_score. I need to research on CDC guidelines


In [11]:
age_list = pd.DataFrame(list(map(lambda x: str_split(x), df.scenario)), columns = ['age','text'])
age_list_2 = nlpCleanup(age_list, 'text')
age_list_lemma = pd.DataFrame(list(map(lambda x: lemmatize(x), age_list_2['text'])), columns = ['text'])
a_l_m =pd.DataFrame((list(map(lambda x: verb_pos_grab(x), age_list_lemma['text']))))

    
first_df = pd.concat([age_list.age,a_l_m], axis = 1) 
first_df.columns = ['age','person','action1']
first_df

,age,person,action1
0,95,woman,enjoy the sunshine in San Francisco at the loc...
1,16,girl with asthma,go to the orthodontist She do not socially dis...
2,gardener,and,cut the grass at the local cemetary He make su...
3,22,man with PTSD,go to a crowd pharmacy No one wear mask and he...
4,26,amazon driver deliver package to,mail room throughout the city they often encou...
...,...,...,...
499,58,woman with blood clot,go to eat lunch in a hospital cafeteria by her...
500,27,massage therapist give massage in a day all of...,wear mask during their hour visit but feet of ...
501,group,friends,go sail in the San Francisco bay None of them ...
502,Lovelace,a tap dance class at his university He,be overweight but wear a fit mask for the enti...


### Sanity Checks and Tests

In [7]:
tester = 'How should I go about this?'
nltk.pos_tag(word_tokenize(tester))

[('How', 'WRB'),
 ('should', 'MD'),
 ('I', 'PRP'),
 ('go', 'VB'),
 ('about', 'IN'),
 ('this', 'DT'),
 ('?', '.')]

### For NLTK Reference

In [8]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or